# Training a Naive Bayes spam filter using scikit-learn

## Load Data

In this step, we load the data from the file. Each sample is a single line in the file and has the following format

*{spam_or_ham},{email_text}*

The first part is the label that identifies whether the email is spam or ham (not spam), followed by the email text. For example:

`Spam,<p>But few feere in nor revellers in pride the a. Ear fathers yes begun revellers blazon one but not of take high. In had his her satiety alone fulness he sins perchance in thence climes nine scorching weary drugged...`

The data will be loaded into two lists. features - the raw text of the emails, and labels - 0 (ham) or 1 (spam)

In [1]:
import random

def read_file(path):
    """
    read and return all data in a file
    """
    with open(path, 'r') as f:
        return f.read()

def load_data():
    """
    load and return the data in features and labels lists
    each item in features contains the raw email text
    each item in labels is either 1(spam) or 0(ham) and identifies corresponding item in features
    """
    # load all data from file
    data_path = "data/SpamDetectionData.txt"
    all_data = read_file(data_path)
    
    # split the data into lines, each line is a single sample
    all_lines = all_data.split('\n')

    # each line in the file is a sample and has the following format
    # it begins with either "Spam," or "Ham,", and follows by the actual text of the email
    # e.g. Spam,<p>His honeyed and land....
    
    # extract the feature (email text) and label (spam or ham) from each line
    features = []
    labels = []
    for line in all_lines:
        if line[0:4] == 'Spam':
            labels.append(1)
            features.append(line[5:])
            pass
        elif line[0:3] == 'Ham':
            labels.append(0)
            features.append(line[4:])
            pass
        else:
            # ignore markers, empty lines and other lines that aren't valid sample
            # print('ignore: "{}"'.format(line));
            pass
    
    return features, labels
    
features, labels = load_data()

print("total no. of samples: {}".format(len(labels)))
print("total no. of spam samples: {}".format(labels.count(1)))
print("total no. of ham samples: {}".format(labels.count(0)))

print("\nPrint a random sample for inspection:")
random_idx = random.randint(0, len(labels))
print("example feature: {}".format(features[random_idx][0:]))
print("example label: {} ({})".format(labels[random_idx], 'spam' if labels[random_idx] else 'ham'))

total no. of samples: 2100
total no. of spam samples: 1043
total no. of ham samples: 1057

Print a random sample for inspection:
example feature: <p>And sought was present loved deem nor near childe the. Now like some prose nor his mother gathered vexed by. Where or old to atonement hall sun the from uncouth sore to delphis his tear lines he he the. She in partings fountain been take like was the mote control was dome almost from deemed lineage congealed. At sick glare was nor sooth her. That in a known sins fondly are times nor him in of rhyme not. At passion into save basked. Crime and that had ear not lone his few strange a. Been hall he ah a for one to. To fabled now.</p><p>Almost adversity the soon he tis seemed carnal sorrow by nine resolved for of spoiled what nor of fall. Hour take glorious native it he dear flee loved like low run formed to hill on childe. Sorrow to come that dares glare oer apart the steel childe awake mine though. From ah aught. Dear yea oh ways. Artless way

## Preprocess Data - Split data randomly into training and test subsets
Use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and test subsets

## Preprocess Data - Vectorize text data
Use [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) to vectorize text input

In [2]:
print([index for index, x in enumerate(labels) if x == 1])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [3]:
from sklearn.model_selection import train_test_split

# load features and labels
features, labels = load_data()

# split data into training / test sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, 
    labels, 
    test_size=0.2,   # use 10% for testing
    random_state=42)

print("no. of train features: {}".format(len(features_train)))
print("no. of train labels: {}".format(len(labels_train)))
print("no. of test features: {}".format(len(features_test)))
print("no. of test labels: {}".format(len(labels_test)))


no. of train features: 1680
no. of train labels: 1680
no. of test features: 420
no. of test labels: 420


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorize email text into tfidf matrix
# TfidfVectorizer converts collection of raw documents to a matrix of TF-IDF features.
# It's equivalent to CountVectorizer followed by TfidfTransformer.
vectorizer = TfidfVectorizer(
    input='content',     # input is actual text
    lowercase=True,      # convert to lower case before tokenizing
    stop_words='english' # remove stop words
)
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed  = vectorizer.transform(features_test)


## Train a Naive Bayes Classifier
Use [MultinomialNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) to train a Naive Bayes classifier

In [5]:
from sklearn.naive_bayes import MultinomialNB
import pickle

def save(vectorizer, classifier):
    '''
    save classifier to disk
    '''
    with open('model.pkl', 'wb') as file:
        pickle.dump((vectorizer, classifier), file)
        
def load():
    '''
    load classifier from disk
    '''
    with open('model.pkl', 'rb') as file:
      vectorizer, classifier = pickle.load(file)
    return vectorizer, classifier

# train a classifier
classifier = MultinomialNB()
classifier.fit(features_train_transformed, labels_train)

# save the trained model
save(vectorizer, classifier)

# score the classifier accuracy
print("classifier accuracy {:.2f}%".format(classifier.score(features_test_transformed, labels_test) * 100))




classifier accuracy 100.00%


# Calculate F1 Score
Calculate [F1 score](https://en.wikipedia.org/wiki/F1_score) using [sklearn metrics.f1_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [6]:
import numpy as np
from sklearn import metrics
prediction = classifier.predict(features_test_transformed)
fscore = metrics.f1_score(labels_test, prediction, average='macro')
print("F score {:.2f}".format(fscore))



F score 1.00


## Using the trained classifier for prediction

In [7]:
vectorizer, classifer = load()

print('\nPerform a test')                    
#email_input = 'enter your email here'
email_input = ['<p>Sick sea he uses might where each sooth would by he and dear friend then. Him this and did virtues it despair given and from be there to things though revel of. Felt charms waste said below breast. Nor haply scorching scorching in sighed vile me he maidens maddest. Alas of deeds monks. Dote my and was sight though. Seemed her feels he childe which care hill.</p><p>Of her was of deigned for vexed given. A along plain. Pile that could can stalked made talethis to of his suffice had. Superstition had losel the formed her of but not knew his departed bliss was the. Riot spent only tear childe. Ere in a disporting more. Of lurked of mine vile be none childe that sore honeyed rill womans she where. She time all upon loathed to known. Seek atonement hall sore where ear. Ofttimes rake domestic dear the monks one thence come friends. A so none climes and kiss prose talethis her when and when then night bidding none childe. Will fame deemed relief delphis he whateer. Soon love scorching low of lone mine ee haply. Than oft lurked worse perchance and gild earth. Are did the losel of none would ofttimes his and. His in this basked such one at so was himnot native. Through though scene and now only hellas but nor later ne but one yet scene yea had.</p>']
email_input_transformed = vectorizer.transform(email_input)
prediction = classifer.predict(email_input_transformed)

print('EMAIL:', email_input)
print('The email is', 'SPAM' if prediction else 'HAM')




Perform a test
EMAIL: ['<p>Sick sea he uses might where each sooth would by he and dear friend then. Him this and did virtues it despair given and from be there to things though revel of. Felt charms waste said below breast. Nor haply scorching scorching in sighed vile me he maidens maddest. Alas of deeds monks. Dote my and was sight though. Seemed her feels he childe which care hill.</p><p>Of her was of deigned for vexed given. A along plain. Pile that could can stalked made talethis to of his suffice had. Superstition had losel the formed her of but not knew his departed bliss was the. Riot spent only tear childe. Ere in a disporting more. Of lurked of mine vile be none childe that sore honeyed rill womans she where. She time all upon loathed to known. Seek atonement hall sore where ear. Ofttimes rake domestic dear the monks one thence come friends. A so none climes and kiss prose talethis her when and when then night bidding none childe. Will fame deemed relief delphis he whateer. 